In [2]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BufferedQueryInterface,QueryTableComponent,BokehTimeseriesGraphic,BokehButton

from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div
from datetime import datetime
import pandas_datareader as pdr

# 6 Basic Dashboard
After exposure to the previous objects, it is possible to bring all the components together into a small dashboard. In this section we will arrange a very basic dashboard using the code that has been written previously.

### Status: In Development
Note, the library is in active development, and virtually not method name or class name, or parameter is stable. 

## 6.1 The Datasoruce
This data source will consolidate all data access for the whole dashboard.



In [4]:

class ExampleComplexQuery(BufferedQueryInterface):

    def appendNextAction(self,indicesIn= None):
        #
        ## First, you are given selected data indices. Sometimes this may be null. in this action we will ignore this index.

        assert indicesIn or not indiciesIn # do something with indices if you want

        #
        ## Second, do something with self.data

        next_day = self.current_date+ timedelta(days=3)
        data=pdr.get_data_yahoo('AAPL',self.current_date,next_day)
        self.data['open_list'].append(data['Open'][0])
        self.data['close_list'].append(data['Close'][0])
        self.data['date_time'].append(next_day)
        assert len(self.data['open_list']) == len(self.data['close_list'])
        assert len(self.data['close_list']) == len(self.data['date_time'])

        self.current_date = next_day 

    def deleteAction(self,indicesIn= None):
        #
        ## Remove all the indices, from largest to smallest
        print("Deleting",indicesIn)
        tables = ['open_list','close_list','date_time']
        indicesIn.sort(reverse=True)
        for i in indicesIn:
            for tableName in tables:
                self.data[tableName].pop(i)

    def load_data_buffer(self):
        try:
            if (self.init):
                return
        except:
            pass
        self.init = 1
        start_init=self._settings['start_date']
        end_init=self._settings['end_date']     
        self.current_date = end_init 
        # Grab data
        start=start_init
        end=end_init 

        data=pdr.get_data_yahoo('AAPL',start,end)
        data_sorted=data.sort_index(axis=0,ascending=True)
        date_list=list(data_sorted.index)

        # Save the data elements
        open_list=list(data_sorted['Open'])
        close_list=list(data_sorted['Close'])
        date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]

        #
        ## First, create the data
        self.data = {
            'open_list':open_list,
            'close_list':close_list,
            'date_time':date_time,
            }

        #
        ## Second, register the append action so it can be used.
        self.registerAction('append',ExampleComplexQuery.appendNextAction)
        self.registerAction('delete',ExampleComplexQuery.deleteAction)
        self.registerAction('kill',ExampleComplexQuery.deleteAction)

# 6.2 All Controls
This is just a very basic button. I will ignore events for now and just focus on layout of the dashboard.

In [5]:
class HelloButton(BokehButton):
    def handle_click(self,event):
        print("Hello I am a button!" + self._settings['name'])

### The Tabs
The most basic analytics applications you will create will likely be broken into tabs. In the folliwng example we illustrate how to create a couple of tabs

In [6]:
class BasicDashboard(Dashboard):
    def getLayout(self):
        # Lets create a dashboard that shows raw data on one tab, and visuals of that data on the other!
        # 
        self.complexQuery = ExampleComplexQuery({'start_date':datetime(2017,4,26),
                                       'end_date':datetime(2017,5,26)})
        settings = {'buffered_query_interface':self.complexQuery,
           'id_field':'open_list',
           'width':600,
           'height':300}

        self.table = QueryTableComponent(settings)
        panel1 = Panel(child=layout([ Div(text="This is the raw data", width=200, height=None),
                                    self.table.getBokehComponent()], sizing_mode='fixed'),title="Raw Data")
        
        # Lets create a second panel that is focused on the visuals
        settings = {'title':'Analytics Plot',
                          'width':600,
                          'height':300,
                          'query':self.complexQuery}

        settings['data_defs'] = [
                    {'y':'open_list','x':'date_time','key':'open','label':'Open','color':'green' },
                    {'y':'close_list','x':'date_time','key':'close','label':'Close','color':'yellow' }                    
                    ]
        
        self.graph = BokehTimeseriesGraphic(settings)        
        self.appendButton = HelloButton({'label':'Hello','name':'After this, we will change data'})
        panel2 = Panel(child=layout([ Div(text="This is the graph", width=200, height=None),self.graph.getBokehComponent()], sizing_mode='fixed'),title="Graph")
        
        # Include panels as tabs, with a control button:        
        tabs = Tabs(tabs=[panel2,panel1], width=700)        
        l = layout([tabs,Div(text="This is below ", width=200, height=None),self.appendButton.getBokehComponent()],sizing_mode='fixed')         
        return l
d = BasicDashboard()
d.showDashboard()

Loading BokehJS ...

Hello I am a button!After this, we will change data
Hello I am a button!After this, we will change data
